# Benchmarking on STM32-Cube.ai



In [1]:
import os, sys, math, datetime
import pathlib
from pathlib import Path

import re
from matplotlib import pyplot as plt
#import plotly.express as px
import pandas as pd
import numpy as np

# import workbench.config.config
from workbench.config.config import initialize
from workbench.utils.utils import create_filepaths
from workbench.wandb import wandb_model_DB, get_model_DB_run_id_from_architecture, get_architecture_from_model_DB_run_id

import wandb
%load_ext autoreload

In [2]:
# Configure pandas to show all columns & rows
pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

In [3]:
models_dir = initialize()

In [4]:
automated = False

global model_name

#model_name = "mobilenetv3smallNSQ_0.25_96_c3_o2_se4l128.MV3"
model_name = "mobilenetv1vvw_0.25_96_c3_o2_vvw"



models_path, models_summary_path, models_image_path, models_layer_df_path, models_tf_path, models_tflite_path, models_tflite_opt_path = create_filepaths(model_name)

c:\tiny_mlc\tiny_cnn\models


In [5]:
models_stm32_benchmark_path = models_dir.joinpath(model_name, f"{model_name}_stm32_benchmark")
models_stm32_benchmark_path

WindowsPath('c:/tiny_mlc/tiny_cnn/models/mobilenetv1vvw_0.25_96_c3_o2_vvw/mobilenetv1vvw_0.25_96_c3_o2_vvw_stm32_benchmark')

In [6]:
#! C:/Users/Susanne/STM32Cube/Repository/Packs/STMicroelectronics/X-CUBE-AI/8.0.0/Utilities/windows/stm32tflm $models_tflite_path

In [7]:
#! C:/Users/Susanne/STM32Cube/Repository/Packs/STMicroelectronics/X-CUBE-AI/8.0.0/Utilities/windows/stm32tfl.exe

In [8]:
#! C:/Users/Susanne/STM32Cube/Repository/Packs/STMicroelectronics/X-CUBE-AI/8.0.0/Utilities/windows/stm32ai 

In [9]:
#! C:/Users/Susanne/STM32Cube/Repository/Packs/STMicroelectronics/X-CUBE-AI/8.0.0/Utilities/windows/stm32ai validate --name network -m $models_tflite_path --type tflite --compression none --verbosity 1 --workspace C:\Users\Susanne\AppData\Local\Temp\mxAI_workspace1067734059012002168110269332871471 --output $models_stm32_benchmark_path --allocate-inputs --allocate-outputs --mode stm32 --desc 115200 --with-report

# Helper functions

In [10]:
def clean_model_summary(filepath): 

    if not filepath.is_file():
        print(f"This file does not exist: {filepath}")
        return None

    else:
        clean_lines = []
        # Parse the MLTK model summary to grab important metrics   
        with open(filepath, "r", encoding="latin-1") as f:
            lines = f.readlines() # list containing lines of file
            for line in lines:
                line = line.strip() # remove leading/trailing white spaces
                if len(line)> 0:
                    
                    clean_lines.append(line)
                else:
                    pass
            #columns = [] # To store column names
        return clean_lines

In [11]:
def clean_column_names(df):
    cols = df.columns

    clean_cols = []
    for col in cols:
        col = col.strip()
        col = col.replace("[" , "")
        col = col.replace("]" , "")   
        clean_cols.append(col)
        
    return clean_cols

In [12]:
def string_percent_to_float(x):
    x = str(x).strip("%")
    return float(x)

In [13]:
def remove_tabs(text):
    """removes tabs from a list of strings

    Args:
        text (list(str)): list of strings that contains tabs

    Returns:
        list(str): list of strings without tabs
    """
    split_text= []
    for l in text:
        split_text.append((l.split("\t")))
        
    return split_text

# Parsing the model file

In [14]:
models_stm32_benchmark_file = models_dir.joinpath(model_name, "network_output", "network_validate_report.txt")
models_stm32_benchmark_file
models_stm32_benchmark_file_INT8 = models_dir.joinpath(model_name, "network_output", "network_validate_report_INT8.txt")
models_stm32_benchmark_file_INT8

WindowsPath('c:/tiny_mlc/tiny_cnn/models/mobilenetv1vvw_0.25_96_c3_o2_vvw/network_output/network_validate_report_INT8.txt')

In [15]:
def clean_benchmark_file(filepath): 

    if not filepath.is_file():
        print(f"This file does not exist: {filepath}")
        return None

    else:
        clean_lines = []
        # Parse the MLTK model summary to grab important metrics   
        with open(filepath, "r", encoding="latin-1") as f:
            lines = f.readlines() # list containing lines of file
            for line in lines:
                #line = line.strip() # remove leading/trailing white spaces
                if len(line)> 0:
                    
                    clean_lines.append(line)
                else:
                    pass
            #columns = [] # To store column names
        return clean_lines

In [16]:
#! explorer $models_path

In [17]:
! code $models_stm32_benchmark_file

In [18]:
! code $models_stm32_benchmark_file_INT8

In [19]:
clean_lines = clean_benchmark_file(models_stm32_benchmark_file_INT8)
#clean_lines

In [20]:
def get_index_of_benchmark_lines(lines):
    model_lines_dict = {}
    for i, line in enumerate(lines):
        if "model name            :" in line:
            model_lines_dict["model_name"] = i
        elif "m_id   layer (type,original)" in line:
            model_lines_dict["m_id"] = i

        elif "model/c-model:" in line:
            model_lines_dict["model/c-model"]=i
            
        elif "activations size" in line:
            model_lines_dict["activations_size"] = i
        elif "weights size" in line:
            model_lines_dict["weights_size"] = i
        elif "macc                  :" in line:
            model_lines_dict["macc"] = i
        elif "duration        :" in line:
            model_lines_dict["duration"] = i
        elif "cycles/MACC     :" in line:
            model_lines_dict["cycles/MACC"] = i
        elif "counter         :" in line:
            model_lines_dict["counter"] = i

        elif "C-Arrays" in line:
            model_lines_dict["c-arrays"] = i
        elif "C-Layers" in line:
            model_lines_dict["c-layers"] = i
        elif "Inference time per node" in line:
            model_lines_dict["inference_times"] = i

        elif "Statistic per tensor" in line:
            model_lines_dict["statistics"] = i
        elif "Number of operations per c-layer" in line:
            model_lines_dict["number_of_operations"] = i
        elif "Number of operation types" in line:
            model_lines_dict["number_of_operation_types"] = i
        elif "Complexity report" in line:
            model_lines_dict["complexity_report"] = i
        elif "Evaluation report " in line:
            model_lines_dict["evaluation_report"] = i
    #print(model_lines_dict)
    return model_lines_dict


In [21]:
model_lines_dict = get_index_of_benchmark_lines(clean_lines)
model_lines_dict

{'m_id': 33,
 'model/c-model': 129,
 'model_name': 135,
 'activations_size': 139,
 'weights_size': 140,
 'macc': 141,
 'c-arrays': 145,
 'c-layers': 293,
 'duration': 559,
 'cycles/MACC': 561,
 'counter': 562,
 'inference_times': 567,
 'statistics': 621,
 'number_of_operations': 653,
 'number_of_operation_types': 706,
 'complexity_report': 717,
 'evaluation_report': 755}

In [22]:
# model_name
name_line = clean_lines[model_lines_dict["model_name"]]
name = name_line.split(":")[1].split()[0].strip()
name

'mobilenetv1vvw_0_25_96_c3_o2_vvw_int8'

In [23]:
suffix=""
inference_stats = {}


In [24]:
if "int8" in name:
    suffix = "_INT8"

In [25]:
# weights_size
activations_line = clean_lines[model_lines_dict["activations_size"]]
activations = int(activations_line.split(":")[1].split()[0].strip())
inference_stats[f"stm32_activations_b{suffix}"]= activations
activations

41152

In [26]:
# weights_size
weights_line = clean_lines[model_lines_dict["weights_size"]]
weights = int(weights_line.split(":")[1].split()[0].strip())
inference_stats[f"stm32_weights_b{suffix}"]= weights
weights

219064

In [27]:
# MACC
macc_line = clean_lines[model_lines_dict["macc"]]
macc = int(macc_line.split(":")[1].strip())
inference_stats[f"stm32_macc_b{suffix}"]= macc
macc

7590360

In [28]:
# cycles/MACC

try:
    cycles_line = clean_lines[model_lines_dict["cycles/MACC"]]
    cycles_line = cycles_line.split(":")[1].strip()
    cycles = float(cycles_line)
    inference_stats[f"stm32_cycles_per_macc_b{suffix}"]= cycles
    cycles
except:
    print("cylces/MACC not reported")

In [29]:
# inference time
duration_line = clean_lines[model_lines_dict["duration"]]
duration_line = duration_line.split(":")[1].strip().split()[0].replace("ms", "")# .map(float)
duration = float(duration_line)
inference_stats[f"stm32_inference_ms{suffix}"]= duration
duration

66.36

In [30]:
inference_stats

{'stm32_activations_b_INT8': 41152,
 'stm32_weights_b_INT8': 219064,
 'stm32_macc_b_INT8': 7590360,
 'stm32_cycles_per_macc_b_INT8': 1.89,
 'stm32_inference_ms_INT8': 66.36}

In [177]:
m_id_lines = clean_lines[model_lines_dict["m_id"]: model_lines_dict["model/c-model"]]
m_id_lines

['m_id   layer (type,original)                        oshape                 param/size           macc               connected to   | c_size      c_macc             c_type                             \n',
 '------ -------------------------------------------- ---------------------- --------------- --------- -------------------------- --- ----------- ------------------ ---------------------------------- \n',
 '0      serving_default_input_10 (Input, )           [b:1,h:96,w:96,c:3]                                                           |             +55,296(+100.0%)   converter_oi8[0]                   \n',
 '       conversion_0 (Conversion, QUANTIZE)          [b:1,h:96,w:96,c:3]                       55,296   serving_default_input_10   |             -55,296(-100.0%)   \n',
 '------ -------------------------------------------- ---------------------- --------------- --------- -------------------------- --- ----------- ------------------ ---------------------------------- \n',
 '1      c

In [178]:
# get inidices for splitting the dataframe
name_indices =[0]
name_indices.append(m_id_lines[0].find("layer (type,original)"))
name_indices.append(m_id_lines[0].find("oshape"))
name_indices.append(m_id_lines[0].find("param/size"))
name_indices.append(m_id_lines[0].find("c_size"))
name_indices.append(m_id_lines[0].find("c_macc"))
name_indices.append(m_id_lines[0].find("c_type"))
name_indices.append(None)
name_indices

[0, 7, 52, 75, 132, 144, 163, None]

In [179]:
# split_indices = [0]
# parts = m_id_lines[1].split(" ")
# print(parts)
# position = 0

# for part in parts:
#     position = position + len(part)
#     split_indices.append(position)
# split_indices.append(None)
# split_indices

In [180]:
#m_id_df.columns

In [181]:
# params_split_df = pd.DataFrame()
# params_split_df["text"] = m_id_df["param/size           macc               connected to   |"].str.strip()#.replace("|", "", inplace=True)#.split(" ",expand=True)
# params_split_df["text"] = params_split_df["text"].str.replace("\|", "")
# params_split_df["connected_to"] = params_split_df["text"].str.split().str[-1]
# params_split_df["macc"] = params_split_df["text"].str.split().str[-2]
# params_split_df["params_size"] = params_split_df["text"].str.split().str[-3]
# params_split_df["params"] = params_split_df["params_size"].str.split("/").str[0]
# params_split_df["size"] = params_split_df["params_size"].str.split("/").str[1]
# params_split_df.head()

In [182]:
suffix

# m_indices = [0, 7, 64, 87, 105, 114, 157, 160, 181, 200, None]
# m_indices_INT8 = [0, 6, 68, 113, 129, 139,  211, 227, 247, None]

# # if suffix =="_INT8":
# #     m_indices = m_indices_INT8

m_indices = name_indices

split_list =[]
for line in m_id_lines[0 :-3]:
    if "---" in line:
        pass
    else:
        split_list.append([line[m_indices[i]:m_indices[i+1]].strip() for i in range(len(m_indices)-1)])
#split_list

m_id_df = pd.DataFrame(split_list)
col_names = m_id_df.iloc[0]
m_id_df = m_id_df[1:]
m_id_df.columns = col_names

m_id_df["m_id"] = m_id_df["m_id"].str.strip()
m_id_df["m_id"].loc[m_id_df["m_id"]==""] = np.NAN
m_id_df["m_id"] = m_id_df["m_id"].fillna(method="ffill")
m_id_df.head()


# split middle part
params_split_df = pd.DataFrame()
params_split_df["text"] = m_id_df["param/size           macc               connected to   |"].str.strip()#.replace("|", "", inplace=True)#.split(" ",expand=True)
params_split_df["text"] = params_split_df["text"].str.replace("\|", "")
params_split_df["connected_to"] = params_split_df["text"].str.split().str[-1]
params_split_df["macc"] = params_split_df["text"].str.split().str[-2]
params_split_df["params_size"] = params_split_df["text"].str.split().str[-3]
params_split_df["params"] = params_split_df["params_size"].str.split("/").str[0]
params_split_df["size"] = params_split_df["params_size"].str.split("/").str[1]
params_split_df.head()


m_id_df["macc"] = params_split_df["macc"]
m_id_df["params"] = params_split_df["params"]
m_id_df["size"] = params_split_df["size"]
m_id_df["connected_to"] = params_split_df["connected_to"]
m_id_df.drop(["param/size           macc               connected to   |"], axis=1, inplace=True)

# clean up macc
m_id_df["macc"].loc[m_id_df["macc"]==""] = np.NAN
m_id_df["macc"] = m_id_df["macc"].str.replace(",", "")
m_id_df["macc"].fillna(0, inplace=True)
m_id_df["macc"] = m_id_df["macc"].astype(int)

m_id_df.drop(m_id_df[m_id_df["layer (type,original)"]==""].index, inplace=True)

# split layer name
layer_split_df = m_id_df["layer (type,original)"].str.split(" \(", expand=True)
layer_split_df[1] = layer_split_df[1].str.replace("\)", "")
#layer_split_df.head()

m_id_df["layer_name"] = layer_split_df[0]
m_id_df["type"] = layer_split_df[1]

m_id_df["params"] = m_id_df["params"].str.replace(",", "")
m_id_df["size"] = m_id_df["size"].str.replace(",", "")
m_id_df["params"].fillna(0, inplace=True)
m_id_df["size"].fillna(0, inplace=True)
m_id_df["params"] = m_id_df["params"].astype(int)
m_id_df["size"] = m_id_df["size"].astype(int)
m_id_df.reset_index(inplace=True, drop=False)

m_id_df.head()

C:\Users\Susanne\AppData\Local\Temp\ipykernel_5420\3899152528.py:33: FutureWarning: The default value of regex will change from True to False in a future version.
  params_split_df["text"] = params_split_df["text"].str.replace("\|", "")
C:\Users\Susanne\AppData\Local\Temp\ipykernel_5420\3899152528.py:58: FutureWarning: The default value of regex will change from True to False in a future version.
  layer_split_df[1] = layer_split_df[1].str.replace("\)", "")


,index,m_id,"layer (type,original)",oshape,c_size,c_macc,c_type,macc,params,size,connected_to,layer_name,type
0,1,0,"serving_default_input_10 (Input, )","[b:1,h:96,w:96,c:3]",,"+55,296(+100.0%)",converter_oi8[0],0,0,0,NaN,serving_default_input_10,"Input,"
1,2,0,"conversion_0 (Conversion, QUANTIZE)","[b:1,h:96,w:96,c:3]",,"-55,296(-100.0%)",,55296,0,0,serving_default_input_10,conversion_0,"Conversion, QUANTIZE"
2,3,1,"conv2d_1 (Conv2D, CONV_2D)","[b:1,h:48,w:48,c:8]",,,optimized_conv2d_oi8[1],497672,224,248,conversion_0,conv2d_1,"Conv2D, CONV_2D"
3,4,1,"nl_1_nl (Nonlinearity, CONV_2D)","[b:1,h:48,w:48,c:8]",,"-18,432(-100.0%)",,18432,0,0,conv2d_1,nl_1_nl,"Nonlinearity, CONV_2D"
4,5,2,"conv2d_2 (Conv2D, DEPTHWISE_CONV_2D)","[b:1,h:48,w:48,c:8]",,,"pad_oi8/conv2d_oi8[2, 3]",165896,80,104,nl_1_nl,conv2d_2,"Conv2D, DEPTHWISE_CONV_2D"


In [183]:
mask = m_id_df["m_id"].duplicated()
s1 = '.' + m_id_df[mask].groupby('m_id').cumcount().add(1).astype(str)
m_id_df.loc[mask, 'm_id'] += s1
#m_id_df["m_id"] = m_id_df["m_id"].astype(float)
m_id_df.head()


,index,m_id,"layer (type,original)",oshape,c_size,c_macc,c_type,macc,params,size,connected_to,layer_name,type
0,1,0,"serving_default_input_10 (Input, )","[b:1,h:96,w:96,c:3]",,"+55,296(+100.0%)",converter_oi8[0],0,0,0,NaN,serving_default_input_10,"Input,"
1,2,0.1,"conversion_0 (Conversion, QUANTIZE)","[b:1,h:96,w:96,c:3]",,"-55,296(-100.0%)",,55296,0,0,serving_default_input_10,conversion_0,"Conversion, QUANTIZE"
2,3,1,"conv2d_1 (Conv2D, CONV_2D)","[b:1,h:48,w:48,c:8]",,,optimized_conv2d_oi8[1],497672,224,248,conversion_0,conv2d_1,"Conv2D, CONV_2D"
3,4,1.1,"nl_1_nl (Nonlinearity, CONV_2D)","[b:1,h:48,w:48,c:8]",,"-18,432(-100.0%)",,18432,0,0,conv2d_1,nl_1_nl,"Nonlinearity, CONV_2D"
4,5,2,"conv2d_2 (Conv2D, DEPTHWISE_CONV_2D)","[b:1,h:48,w:48,c:8]",,,"pad_oi8/conv2d_oi8[2, 3]",165896,80,104,nl_1_nl,conv2d_2,"Conv2D, DEPTHWISE_CONV_2D"


In [184]:
#m_id_df.drop(m_id_df[m_id_df["layer (type,original)"]==""].index, inplace=True)

In [185]:
m_id_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60 entries, 0 to 59
Data columns (total 13 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   index                  60 non-null     int64 
 1   m_id                   60 non-null     object
 2   layer (type,original)  60 non-null     object
 3   oshape                 60 non-null     object
 4   c_size                 60 non-null     object
 5   c_macc                 60 non-null     object
 6   c_type                 60 non-null     object
 7   macc                   60 non-null     int32 
 8   params                 60 non-null     int32 
 9   size                   60 non-null     int32 
 10  connected_to           59 non-null     object
 11  layer_name             60 non-null     object
 12  type                   60 non-null     object
dtypes: int32(3), int64(1), object(9)
memory usage: 5.5+ KB


In [186]:
#m_id_df.to_csv("m_id_df.csv")

In [187]:
# m_id_df_slimm = m_id_df.copy()
# m_id_df_slimm.drop(m_id_df_slimm[m_id_df_slimm.type=="Input, "].index, inplace=True)
# m_id_df_slimm.drop(m_id_df_slimm[m_id_df_slimm.type=="Placeholder, "].index, inplace=True)
# m_id_df_slimm.drop(m_id_df_slimm[m_id_df_slimm.type=="Pad, PAD"].index, inplace=True)
# m_id_df_slimm.drop(m_id_df_slimm[m_id_df_slimm.type==""].index, inplace=True)
# m_id_df_slimm.drop(m_id_df_slimm[m_id_df_slimm.type=="Nonlinearity, HARD_SWISH"].index, inplace=True)
# m_id_df_slimm.drop(m_id_df_slimm[m_id_df_slimm.type=="Nonlinearity, DEPTHWISE_CONV_2D"].index, inplace=True)
# m_id_df_slimm.dropna(subset=["type"], inplace=True)
# m_id_df_slimm.drop(m_id_df_slimm[m_id_df_slimm.type=="Nonlinearity, CONV_2D"].index, inplace=True)
# #m_id_df_slimm.drop(m_id_df_slimm[m_id_df_slimm.type is None].index, inplace=True)
# m_id_df_slimm.reset_index(inplace=True)

# m_id_df_slimm.index +=1
# m_id_df_slimm

# Getting inference times per layer

In [188]:
inference_lines = clean_lines[model_lines_dict["inference_times"]: model_lines_dict["statistics"]]
inference_lines

['   Inference time per node\n',
 '   --------------------------------------------------------------------------------------------\n',
 '   c_id    m_id   type                 dur (ms)     %         counters            name         \n',
 '   --------------------------------------------------------------------------------------------\n',
 '   0       0      NL (0x107)                0.291      0.4%   [         62,794]   ai_node_0    \n',
 '   1       1      Conv2dPool (0x109)        7.416     11.2%   [      1,601,784]   ai_node_1    \n',
 '   2       2      Pad (0x116)               0.071      0.1%   [         15,426]   ai_node_2    \n',
 '   3       2      Conv2D (0x103)            3.422      5.2%   [        739,097]   ai_node_3    \n',
 '   4       3      Conv2D (0x103)            4.343      6.5%   [        938,079]   ai_node_4    \n',
 '   5       4      Pad (0x116)               0.111      0.2%   [         23,969]   ai_node_5    \n',
 '   6       4      Conv2D (0x103)            1.6

In [189]:
def string_percent_to_float(x):
    x = str(x).strip("%")
    return float(x)

In [190]:
indices = [0, 8, 15,36,49, 60,79, None]
split_list =[]
for line in inference_lines[2 :-3]:
    if "---" in line:
        pass
    else:
        split_list.append([line[indices[i]:indices[i+1]].strip() for i in range(len(indices)-1)])

inference_df = pd.DataFrame(split_list)
col_names = inference_df.iloc[0]
inference_df = inference_df[1:]
inference_df.columns = col_names
inference_df["c_id"] = inference_df["c_id"].str.strip()
inference_df["dur (ms)"] = inference_df["dur (ms)"].astype(float)
inference_df["%"] = inference_df["%"].apply(string_percent_to_float)
inference_df["counters"] = inference_df["counters"].str.replace("[", "")
inference_df["counters"] = inference_df["counters"].str.replace("]", "")
inference_df["counters"] = inference_df["counters"].str.replace(",", "").astype(int)
inference_df.rename(columns={"name": "node_name", "type": "node_type"}, errors="raise", inplace=True)
#print(col_names)
inference_df

C:\Users\Susanne\AppData\Local\Temp\ipykernel_5420\1061571099.py:16: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  inference_df["counters"] = inference_df["counters"].str.replace("[", "")
C:\Users\Susanne\AppData\Local\Temp\ipykernel_5420\1061571099.py:17: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  inference_df["counters"] = inference_df["counters"].str.replace("]", "")


,c_id,m_id,node_type,dur (ms),%,counters,node_name
1,0,0,NL (0x107),0.291,0.4,62794,ai_node_0
2,1,1,Conv2dPool (0x109),7.416,11.2,1601784,ai_node_1
3,2,2,Pad (0x116),0.071,0.1,15426,ai_node_2
4,3,2,Conv2D (0x103),3.422,5.2,739097,ai_node_3
5,4,3,Conv2D (0x103),4.343,6.5,938079,ai_node_4
6,5,4,Pad (0x116),0.111,0.2,23969,ai_node_5
7,6,4,Conv2D (0x103),1.661,2.5,358843,ai_node_6
8,7,5,Conv2D (0x103),2.353,3.5,508156,ai_node_7
9,8,6,Pad (0x116),0.059,0.1,12834,ai_node_8
10,9,6,Conv2D (0x103),3.233,4.9,698410,ai_node_9


In [191]:
inference_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46 entries, 1 to 46
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   c_id       46 non-null     object 
 1   m_id       46 non-null     object 
 2   node_type  46 non-null     object 
 3   dur (ms)   46 non-null     float64
 4   %          46 non-null     float64
 5   counters   46 non-null     int32  
 6   node_name  46 non-null     object 
dtypes: float64(2), int32(1), object(4)
memory usage: 2.5+ KB


In [192]:
#inference_df.to_csv("inference_df.csv")

In [193]:
merged_df = pd.merge(inference_df,m_id_df,on='m_id', how="outer")#inference_df.join(m_id_df_slimm, on="m_id", how="left")
merged_df#.head(20)

,c_id,m_id,node_type,dur (ms),%,counters,node_name,index,"layer (type,original)",oshape,c_size,c_macc,c_type,macc,params,size,connected_to,layer_name,type
0,0,0,NL (0x107),0.291,0.4,62794.0,ai_node_0,1,"serving_default_input_10 (Input, )","[b:1,h:96,w:96,c:3]",,"+55,296(+100.0%)",converter_oi8[0],0,0,0,NaN,serving_default_input_10,"Input,"
1,1,1,Conv2dPool (0x109),7.416,11.2,1601784.0,ai_node_1,3,"conv2d_1 (Conv2D, CONV_2D)","[b:1,h:48,w:48,c:8]",,,optimized_conv2d_oi8[1],497672,224,248,conversion_0,conv2d_1,"Conv2D, CONV_2D"
2,2,2,Pad (0x116),0.071,0.1,15426.0,ai_node_2,5,"conv2d_2 (Conv2D, DEPTHWISE_CONV_2D)","[b:1,h:48,w:48,c:8]",,,"pad_oi8/conv2d_oi8[2, 3]",165896,80,104,nl_1_nl,conv2d_2,"Conv2D, DEPTHWISE_CONV_2D"
3,3,2,Conv2D (0x103),3.422,5.2,739097.0,ai_node_3,5,"conv2d_2 (Conv2D, DEPTHWISE_CONV_2D)","[b:1,h:48,w:48,c:8]",,,"pad_oi8/conv2d_oi8[2, 3]",165896,80,104,nl_1_nl,conv2d_2,"Conv2D, DEPTHWISE_CONV_2D"
4,4,3,Conv2D (0x103),4.343,6.5,938079.0,ai_node_4,7,"conv2d_3 (Conv2D, CONV_2D)","[b:1,h:48,w:48,c:16]",,,conv2d_oi8[4],294928,144,192,nl_2_nl,conv2d_3,"Conv2D, CONV_2D"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,NaN,24.1,NaN,NaN,NaN,NaN,NaN,50,"nl_24_nl (Nonlinearity, DEPTHWISE_CONV_2D)","[b:1,h:3,w:3,c:128]",,"-1,152(-100.0%)",,1152,0,0,conv2d_24,nl_24_nl,"Nonlinearity, DEPTHWISE_CONV_2D"
71,NaN,25.1,NaN,NaN,NaN,NaN,NaN,52,"nl_25_nl (Nonlinearity, CONV_2D)","[b:1,h:3,w:3,c:256]",,"-2,304(-100.0%)",,2304,0,0,conv2d_25,nl_25_nl,"Nonlinearity, CONV_2D"
72,NaN,26.1,NaN,NaN,NaN,NaN,NaN,54,"nl_26_nl (Nonlinearity, DEPTHWISE_CONV_2D)","[b:1,h:3,w:3,c:256]",,"-2,304(-100.0%)",,2304,0,0,conv2d_26,nl_26_nl,"Nonlinearity, DEPTHWISE_CONV_2D"
73,NaN,27.1,NaN,NaN,NaN,NaN,NaN,56,"nl_27_nl (Nonlinearity, CONV_2D)","[b:1,h:3,w:3,c:256]",,"-2,304(-100.0%)",,2304,0,0,conv2d_27,nl_27_nl,"Nonlinearity, CONV_2D"


In [162]:
# merged_df.drop(merged_df[merged_df.type=="Nonlinearity, HARD_SWISH"].index, inplace=True)
# merged_df.drop(merged_df[merged_df.type=="Nonlinearity, DEPTHWISE_CONV_2D"].index, inplace=True)

# merged_df.drop(merged_df[merged_df.type=="Nonlinearity, CONV_2D"].index, inplace=True)
# merged_df.drop(merged_df[merged_df.type=="Eltwise, MINIMUM"].index, inplace=True)

# merged_df.drop(merged_df[merged_df.type=="Nonlinearity, RELU"].index, inplace=True)
# #merged_df.drop(merged_df[merged_df["layer (type,original)"=="Nonlinearity, RELU"].index, inplace=True)
# merged_df.dropna(subset=["layer (type,original)"], inplace=True)
# merged_df.dropna(subset=["c_id"], inplace=True)
# merged_df["type"].fillna("", inplace=True)
# merged_df.drop_duplicates(subset=["layer (type,original)"], keep="first", inplace=True)
# #merged_df.drop("index", axis=1, inplace=True)
# merged_df.reset_index(inplace=True, drop=False)
# merged_df.rename(columns={"index": "_index"}, errors="raise", inplace=True)


In [194]:
merged_df.columns

Index(['c_id', 'm_id', 'node_type', 'dur (ms)', '%', 'counters', 'node_name',
       'index', 'layer (type,original)', 'oshape', 'c_size', 'c_macc',
       'c_type', 'macc', 'params', 'size', 'connected_to', 'layer_name',
       'type'],
      dtype='object', name=0)

In [195]:
merged_df.isnull().sum()

0
c_id                     29
m_id                      0
node_type                29
dur (ms)                 29
%                        29
counters                 29
node_name                29
index                     0
layer (type,original)     0
oshape                    0
c_size                    0
c_macc                    0
c_type                    0
macc                      0
params                    0
size                      0
connected_to              1
layer_name                0
type                      0
dtype: int64

In [204]:
merged_df["m_id"] =  merged_df["m_id"].astype(float)
merged_df.sort_values("m_id", inplace=True)
merged_df.reset_index(inplace=True, drop=True)
merged_df.fillna(0, inplace=True)
merged_df.head()

,level_0,c_id,m_id,node_type,dur (ms),%,counters,node_name,index,"layer (type,original)",oshape,c_size,c_macc,c_type,macc,params,size,connected_to,layer_name,type
0,0,0,0.0,NL (0x107),0.291,0.4,62794.0,ai_node_0,1,"serving_default_input_10 (Input, )","[b:1,h:96,w:96,c:3]",,"+55,296(+100.0%)",converter_oi8[0],0,0,0,0,serving_default_input_10,"Input,"
1,46,0,0.1,0,0.000,0.0,0.0,0,2,"conversion_0 (Conversion, QUANTIZE)","[b:1,h:96,w:96,c:3]",,"-55,296(-100.0%)",,55296,0,0,serving_default_input_10,conversion_0,"Conversion, QUANTIZE"
2,1,1,1.0,Conv2dPool (0x109),7.416,11.2,1601784.0,ai_node_1,3,"conv2d_1 (Conv2D, CONV_2D)","[b:1,h:48,w:48,c:8]",,,optimized_conv2d_oi8[1],497672,224,248,conversion_0,conv2d_1,"Conv2D, CONV_2D"
3,47,0,1.1,0,0.000,0.0,0.0,0,4,"nl_1_nl (Nonlinearity, CONV_2D)","[b:1,h:48,w:48,c:8]",,"-18,432(-100.0%)",,18432,0,0,conv2d_1,nl_1_nl,"Nonlinearity, CONV_2D"
4,2,2,2.0,Pad (0x116),0.071,0.1,15426.0,ai_node_2,5,"conv2d_2 (Conv2D, DEPTHWISE_CONV_2D)","[b:1,h:48,w:48,c:8]",,,"pad_oi8/conv2d_oi8[2, 3]",165896,80,104,nl_1_nl,conv2d_2,"Conv2D, DEPTHWISE_CONV_2D"


In [198]:
# combined_df = m_id_df_slimm.copy()
# combined_df["c_type"] = inference_df["type"]
# combined_df["dur (ms)"] = inference_df["dur (ms)"]
# combined_df["counters"] = inference_df["counters"]
# combined_df["node_name"] = inference_df["node_name"]
# combined_df

In [ ]:
#combined_df[["m_id", "layer_name", "type_x", "type_y", "dur (ms)"]]

In [ ]:
#inference_df["c_id"].unique()

# Logging to wandb

In [ ]:
# Generate run ids
#id = wandb.wandb.sdk.lib.runid.generate_id()

id = get_model_DB_run_id_from_architecture(model_name)
PROJECT = "model_DB"

run = wandb.init(
        # Set the project where this run will be logged
        project=PROJECT, 
        id = id, 
        resume="allow",
        )


# # Create a table
stm32_inference_df_table = wandb.Table(dataframe=inference_df)
stm32_int8_table = wandb.Table(dataframe= merged_df)

run.log({"stm32_inference_times_int8": stm32_inference_df_table})
run.log({"stm32_inference_int8": stm32_int8_table})

run.log(inference_stats)



wandb.finish()